In [1]:
#Importanción de librerías
import os
#import argparse
#import matplotlib
import numpy as np
import math
from plantcv.parallel import WorkflowInputs
from plantcv import plantcv as pcv
import pandas as pd

# HSV and CIE-Lab from RGB values
from utilities import convert_rgb_to_lab_hsv

In [2]:
# Define the column titles
column_titles = ['archivo', 'largo_mm', 'ancho_mm', 'area_mm2', 'perimetro_mm', 'solidez','redondez',
                'R','G', 'B', 'CIE_L','CIE_a','CIE_b','H','S', 'V']


# Create an empty DataFrame with the specified columns
data = pd.DataFrame(columns=column_titles)


# Define the directory containing images
image_directory = './img'

In [6]:
# Process each image in a directory
for filename in os.listdir(image_directory):
    if filename.startswith('._'):
        os.remove(os.path.join(image_directory, filename))
    else:

        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other image formats as needed
            image_path = os.path.join(image_directory, filename)
            img, path, filename = pcv.readimage(filename=image_path)
            rotate_img=pcv.transform.rotate(img=img,rotation_deg=90,crop=False)
            card_mask = pcv.transform.detect_color_card(rgb_img=rotate_img, adaptive_method=1, block_size=51, radius=30)
            headers, card_matrix = pcv.transform.get_color_matrix(rgb_img=rotate_img, mask=card_mask)
        
            std_color_matrix = pcv.transform.std_color_matrix(pos=3)
            
            img_cc = pcv.transform.affine_color_correction(rgb_img=rotate_img, 
                                                           source_matrix=card_matrix,
                                                           target_matrix=std_color_matrix)
            b_img = pcv.rgb2gray_lab(rgb_img=img_cc, channel='b')
            th_otsu=pcv.threshold.otsu(gray_img=b_img, object_type='light')
            a_erode=pcv.erode(gray_img=th_otsu, ksize=2, i=1)
            a_dilation=pcv.dilate(gray_img=a_erode, ksize=2, i=1)
            mask_fill = pcv.fill(bin_img=a_dilation, size=2000)
            mask_fill = pcv.fill_holes(bin_img=mask_fill)
            roi1 = pcv.roi.rectangle(img=img_cc, x=450, y=380, h=2000, w=1960)
            kept_mask  = pcv.roi.filter(mask=mask_fill, roi=roi1, roi_type='partial')
            analysis_image = pcv.analyze.size(img=img_cc, labeled_mask=kept_mask)
            masked_plant = pcv.apply_mask(img=img_cc, mask=kept_mask, mask_color='white')
        
            #Manually Obtain morphological values 
            chip_length=12 #mm
            
            avg_chip_size = pcv.outputs.metadata['median_color_chip_size']['value'][0]
            chip_width_px = pcv.outputs.metadata['median_color_chip_width']['value'][0]  # px
            chip_height_px = pcv.outputs.metadata['median_color_chip_height']['value'][0]  # px
            chip_length_avg=(chip_width_px+chip_height_px)/2 # cm
            ratio_length=chip_length/chip_length_avg 
            
            ratio_area=(chip_length**2)/chip_length_avg**2
        
            # Simulación de análisis de semillas (código de ejemplo para demostrar funcionalidad)
            
            height = pcv.outputs.observations['default_1']['height']['value'] * ratio_length
            width = pcv.outputs.observations['default_1']['width']['value'] * ratio_length
            area = pcv.outputs.observations['default_1']['area']['value'] * ratio_area
            perimeter = pcv.outputs.observations['default_1']['perimeter']['value'] * ratio_length
            solidity = pcv.outputs.observations['default_1']['solidity']['value']  # mide qué tan redondo es un objeto
            roundness= 4*math.pi*area/perimeter**2
            list=[filename, height, width, area, perimeter, solidity,roundness] 
            
            
            #Color
            #Proceso para obtener los píxeles que corresponden a la superficie de la planta
            nrows, ncolumns, nlayers = masked_plant.shape
            
            
            #Dimensión de la imagen
            dim_img=nrows*ncolumns
            
            #Conversión de la matriz en un vector
            vector_plant=np.reshape(masked_plant,(nrows*ncolumns, -1))
            
            #Obtención de píxeles de color
            RGBplant=vector_plant[vector_plant[:,0]<255]
            
            b_plant_prom=RGBplant[:,0].mean()
            g_plant_prom=RGBplant[:,1].mean()
            r_plant_prom=RGBplant[:,2].mean()
            
            # Conversión a los espacios CIE-Lab, y HSV
            #Obtenemos los valores promedio RGB de la hoja
            rgb_triplet = (r_plant_prom,g_plant_prom,b_plant_prom) 
            
            #Find lab and hsv color model channels using convert_rgb_to_lab_hsv, numpy and opencv
            lab_color, hsv_color = convert_rgb_to_lab_hsv(rgb_triplet)
            
            #lists conversion from matrix
            lab=lab_color.tolist()
            hsv=hsv_color.tolist()
            
            list.extend([r_plant_prom, g_plant_prom, b_plant_prom, lab[0],lab[1],lab[2],hsv[0],hsv[1], hsv[2]])
            
            # Agrega los datos al DataFrame
            data.loc[len(data)]= list



In [ ]:
#Opcional para ver los objetos y resultados del análisis con la función pcv.analyze.size 
# Save out data to file
pcv.outputs.save_results(filename="./res/results_analyze_size.txt", outformat="json")

In [7]:
# Define el archivo de salida
archivo = './res/res_hojas.xlsx'

# Crea el directorio si no existe
directory = os.path.dirname(archivo)
if directory and not os.path.exists(directory):
    os.makedirs(directory, exist_ok=True)

# Convert columns to numeric, forcing errors to NaN, then round
for col in ['largo_mm', 'ancho_mm', 'area_mm2', 'perimetro_mm', 'solidez','redondez']:
    data[col] = pd.to_numeric(data[col], errors='coerce').map(lambda x: round(x, 2) if pd.notnull(x) else x)

# Check if the file exists
if os.path.exists(archivo):
    # If the file exists, load the workbook and append to the 'Results' sheet if it exists
    with pd.ExcelWriter(archivo, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
        # Check if 'Results' sheet already exists
        if 'Results' in writer.book.sheetnames:
            # Get the maximum row in the existing sheet to append the new data below it
            startrow = writer.sheets['Results'].max_row
        else:
            # If the 'Results' sheet does not exist, start from the first row
            startrow = 0
        # Write the DataFrame to the existing file, appending data if the sheet exists <----- Key procedure!
        data.to_excel(writer, sheet_name='Results', header=startrow == 0, startrow=startrow, index=True)
else:
    # If the file does not exist, create it and write the data
    with pd.ExcelWriter(archivo, engine='openpyxl') as writer:
        data.to_excel(writer, sheet_name='Results', index=True)
